In [1]:
import xarray as xr
from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
import json
import ssl
import itertools
from urllib.parse import urlparse
from getpass import getpass
import base64
from urllib.error import HTTPError, URLError
import os
import math
import time
import sys
import shutil
from nsidc_functions import *

In [2]:
domain_id = 3
trial_id = 87
domains_direc = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/ASO/selected/'
pathh = '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/VIIRS/'
pathh += 'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'/'
if not os.path.exists(pathh):
    os.makedirs(pathh)
else:
    print(f"Directory '{pathh}' already exists.")

time_start = '2022-09-01T00:00:00Z'
time_end = '2023-09-01T00:00:00Z'
buffer = 0.5

short_name = 'VNP10A1F'
version = '2'

# #### GLOBAL PARAMETERS FOR THE DOWNSCALING SCRIPTS
# polygon = ''
# filename_filter = ''
# quiet = False
# force = False
# CMR_URL = 'https://cmr.earthdata.nasa.gov'
# CMR_COLLECTIONS_URL = '{0}/search/collections.json?'.format(CMR_URL)
# CMR_URL = 'https://cmr.earthdata.nasa.gov'
# CMR_PAGE_SIZE = 2000
# CMR_FILE_URL = ('{0}/search/granules.json?'
#                 '&sort_key[]=start_date&sort_key[]=producer_granule_id'
#                 '&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))
# FILE_DOWNLOAD_MAX_RETRIES = 3

Directory '/Users/jpflug/Documents/Projects/snowCover_comparisons/Data/VIIRS/domain03_trial87/' already exists.


In [3]:
dom_ref = domains_direc+'domain'+str(domain_id).zfill(2)+'_trial'+str(trial_id).zfill(2)+'.tif'
dom_ref = xr.open_dataset(dom_ref).isel(band=0).rio.reproject(4326)

x_lims = [dom_ref.x.mean().values-buffer,dom_ref.x.mean().values+buffer]
y_lims = [dom_ref.y.mean().values-buffer,dom_ref.y.mean().values+buffer]
bounding_box = str(x_lims[0])+','+str(y_lims[0])+','+str(x_lims[1])+','+str(y_lims[1])
print(x_lims,y_lims)
print(bounding_box)

[-106.41083996827871, -105.41083996827871] [39.0839743260506, 40.0839743260506]
-106.41083996827871,39.0839743260506,-105.41083996827871,40.0839743260506


In [4]:
def cmr_download(urls, pathh, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = url.split('/')[-1]
        print(filename)
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        for download_attempt_number in range(1, FILE_DOWNLOAD_MAX_RETRIES + 1):
            if not quiet and download_attempt_number > 1:
                print('Retrying download of {0}'.format(url))
            try:
                response = get_login_response(url, credentials, token)
                length = int(response.headers['content-length'])
                try:
                    if not force and length == os.path.getsize(filename):
                        if not quiet:
                            print('  File exists, skipping')
                        # We have already downloaded the file. Break out of the
                        # retry loop.
                        break
                except OSError:
                    pass
                count = 0
                chunk_size = min(max(length, 1), 1024 * 1024)
                max_chunks = int(math.ceil(length / chunk_size))
                time_initial = time.time()
                with open(filename, 'wb') as out_file:
                    for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                        out_file.write(data)
#                         shutil.copy(filename,pathh+filename)
#                         os.remove(filename)
                        if not quiet:
                            count = count + 1
                            time_elapsed = time.time() - time_initial
                            download_speed = get_speed(time_elapsed, count * chunk_size)
                            output_progress(count, max_chunks, status=download_speed)
                if not quiet:
                    print()
                # If we get here, the download was successful and we can break
                # out of the retry loop.
                break
            except HTTPError as e:
                print('HTTP error {0}, {1}'.format(e.code, e.reason))
            except URLError as e:
                print('URL error: {0}'.format(e.reason))
            except IOError:
                raise

            # If this happens, none of our attempts to download the file
            # succeeded. Print an error message and raise an error.
            if download_attempt_number == FILE_DOWNLOAD_MAX_RETRIES:
                print('failed to download file {0}.'.format(filename))
                sys.exit(1)

In [5]:
url_list = cmr_search(short_name, version, time_start, time_end,
                      bounding_box=bounding_box, polygon=polygon,
                      filename_filter=filename_filter, quiet=quiet)

Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?&sort_key[]=start_date&sort_key[]=producer_granule_id&page_size=2000&short_name=VNP10A1F&version=002&version=02&version=2&temporal[]=2022-09-01T00:00:00Z,2023-09-01T00:00:00Z&bounding_box=-106.41083996827871,39.0839743260506,-105.41083996827871,40.0839743260506&provider=NSIDC_ECS

Found 732 matches.


In [ ]:
cmr_download(url_list, pathh, force=force, quiet=quiet)

Earthdata username (or press Return to use a bearer token): jpflug
password: ········
VNP10A1F.A2022244.h09v04.002.2023056014856.h5
0001/1464: VNP10A1F.A2022244.h09v04.002.2023056014856.h5
  [============================================================] 100%  147.8kB/s   
VNP10A1F.A2022244.h09v04.002.2023056014856.h5.xml
0002/1464: VNP10A1F.A2022244.h09v04.002.2023056014856.h5.xml
  [============================================================] 100%  3.0MB/s   
VNP10A1F.A2022244.h09v05.002.2023056014852.h5
0003/1464: VNP10A1F.A2022244.h09v05.002.2023056014852.h5
  [====================                                        ]  33%  1.5MB/s    [========================================                    ]  67%  2.1MB/s    [============================================================] 100%  2.8MB/s   
VNP10A1F.A2022244.h09v05.002.2023056014852.h5.xml
0004/1464: VNP10A1F.A2022244.h09v05.002.2023056014852.h5.xml
  [============================================================] 100%  3.2

  [============================================================] 100%  1.9MB/s   
VNP10A1F.A2022253.h09v04.002.2023056214031.h5
0037/1464: VNP10A1F.A2022253.h09v04.002.2023056214031.h5
  [==============================                              ]  50%  216.2kB/s    [============================================================] 100%  416.4kB/s   
VNP10A1F.A2022253.h09v04.002.2023056214031.h5.xml
0038/1464: VNP10A1F.A2022253.h09v04.002.2023056214031.h5.xml
  [============================================================] 100%  2.4MB/s   
VNP10A1F.A2022253.h09v05.002.2023056212312.h5
0039/1464: VNP10A1F.A2022253.h09v05.002.2023056212312.h5
  [==============================                              ]  50%  1.5MB/s    [============================================================] 100%  2.2MB/s   
VNP10A1F.A2022253.h09v05.002.2023056212312.h5.xml
0040/1464: VNP10A1F.A2022253.h09v05.002.2023056212312.h5.xml
  [============================================================] 100%  2.0M

  [====================                                        ]  33%  1.3MB/s    [========================================                    ]  67%  1.7MB/s    [============================================================] 100%  2.4MB/s   
VNP10A1F.A2022261.h09v04.002.2023057134237.h5.xml
0070/1464: VNP10A1F.A2022261.h09v04.002.2023057134237.h5.xml
  [============================================================] 100%  3.5MB/s   
VNP10A1F.A2022261.h09v05.002.2023057134228.h5
0071/1464: VNP10A1F.A2022261.h09v05.002.2023057134228.h5
  [==============================                              ]  50%  1.5MB/s    [============================================================] 100%  2.8MB/s   
VNP10A1F.A2022261.h09v05.002.2023057134228.h5.xml
0072/1464: VNP10A1F.A2022261.h09v05.002.2023057134228.h5.xml
  [============================================================] 100%  2.2MB/s   
VNP10A1F.A2022262.h09v04.002.2023057160105.h5
0073/1464: VNP10A1F.A2022262.h09v04.002.2023057160105.h5


  [============================================================] 100%  2.7MB/s   
VNP10A1F.A2022270.h09v05.002.2023058075945.h5
0107/1464: VNP10A1F.A2022270.h09v05.002.2023058075945.h5
  [==============================                              ]  50%  1.6MB/s    [============================================================] 100%  2.8MB/s   
VNP10A1F.A2022270.h09v05.002.2023058075945.h5.xml
0108/1464: VNP10A1F.A2022270.h09v05.002.2023058075945.h5.xml
  [============================================================] 100%  3.0MB/s   
VNP10A1F.A2022271.h09v04.002.2023058092320.h5
0109/1464: VNP10A1F.A2022271.h09v04.002.2023058092320.h5
  [==============================                              ]  50%  1.5MB/s    [============================================================] 100%  2.3MB/s   
VNP10A1F.A2022271.h09v04.002.2023058092320.h5.xml
0110/1464: VNP10A1F.A2022271.h09v04.002.2023058092320.h5.xml
  [============================================================] 100%  2.6MB/s 

  [============================================================] 100%  379.0kB/s   
VNP10A1F.A2022279.h09v04.002.2023067192202.h5.xml
0142/1464: VNP10A1F.A2022279.h09v04.002.2023067192202.h5.xml
  [============================================================] 100%  2.8MB/s   
VNP10A1F.A2022279.h09v05.002.2023067191232.h5
0143/1464: VNP10A1F.A2022279.h09v05.002.2023067191232.h5
  [====================                                        ]  33%  895.3kB/s    [========================================                    ]  67%  900.2kB/s    [============================================================] 100%  1.1MB/s   
VNP10A1F.A2022279.h09v05.002.2023067191232.h5.xml
0144/1464: VNP10A1F.A2022279.h09v05.002.2023067191232.h5.xml
  [============================================================] 100%  2.6MB/s   
VNP10A1F.A2022280.h09v04.002.2023067212406.h5
0145/1464: VNP10A1F.A2022280.h09v04.002.2023067212406.h5
  [==============================                              ]  50%  81

...

  [========                                                    ]  12%  271.4kB/s    [===============                                             ]  25%  389.7kB/s    [======================                                      ]  38%  505.4kB/s    [==============================                              ]  50%  611.5kB/s    [======================================                      ]  62%  709.6kB/s    [=============================================               ]  75%  800.2kB/s    [====================================================        ]  88%  887.2kB/s    [============================================================] 100%  1.0MB/s   
VNP10A1F.A2022365.h09v04.002.2023089122846.h5.xml
0486/1464: VNP10A1F.A2022365.h09v04.002.2023089122846.h5.xml
  [============================================================] 100%  3.0MB/s   
VNP10A1F.A2022365.h09v05.002.2023089122841.h5
0487/1464: VNP10A1F.A2022365.h09v05.002.2023089122841.h5
  [============                              

  [=========                                                   ]  14%  276.5kB/s    [=================                                           ]  29%  404.2kB/s    [==========================                                  ]  43%  516.0kB/s    [==================================                          ]  57%  622.1kB/s    [===========================================                 ]  71%  725.7kB/s    [===================================================         ]  86%  806.1kB/s    [============================================================] 100%  903.3kB/s   
VNP10A1F.A2023005.h09v04.002.2023093225458.h5.xml
0506/1464: VNP10A1F.A2023005.h09v04.002.2023093225458.h5.xml
  [============================================================] 100%  1.7MB/s   
VNP10A1F.A2023005.h09v05.002.2023093224804.h5
0507/1464: VNP10A1F.A2023005.h09v05.002.2023093224804.h5
  [============                                                ]  20%  291.0kB/s    [========================                

  [=========                                                   ]  14%  267.5kB/s    [=================                                           ]  29%  391.7kB/s    [==========================                                  ]  43%  502.8kB/s    [==================================                          ]  57%  607.8kB/s    [===========================================                 ]  71%  698.1kB/s    [===================================================         ]  86%  784.8kB/s    [============================================================] 100%  882.5kB/s   
VNP10A1F.A2023010.h09v04.002.2023094095630.h5.xml
0526/1464: VNP10A1F.A2023010.h09v04.002.2023094095630.h5.xml
  [============================================================] 100%  3.1MB/s   
VNP10A1F.A2023010.h09v05.002.2023094093929.h5
0527/1464: VNP10A1F.A2023010.h09v05.002.2023094093929.h5
  [===============                                             ]  25%  279.6kB/s    [==============================          

  [============================================================] 100%  2.2MB/s   
VNP10A1F.A2023015.h09v05.002.2023095233013.h5
0547/1464: VNP10A1F.A2023015.h09v05.002.2023095233013.h5
  [===============                                             ]  25%  266.1kB/s    [==============================                              ]  50%  382.2kB/s    [=============================================               ]  75%  483.9kB/s    [============================================================] 100%  613.1kB/s   
VNP10A1F.A2023015.h09v05.002.2023095233013.h5.xml
0548/1464: VNP10A1F.A2023015.h09v05.002.2023095233013.h5.xml
  [============================================================] 100%  3.2MB/s   
VNP10A1F.A2023016.h09v04.002.2023096042420.h5
0549/1464: VNP10A1F.A2023016.h09v04.002.2023096042420.h5
  [========                                                    ]  12%  269.5kB/s    [===============                                             ]  25%  395.7kB/s    [==================

  [============                                                ]  20%  257.7kB/s    [========================                                    ]  40%  375.4kB/s    [====================================                        ]  60%  483.9kB/s    [================================================            ]  80%  584.3kB/s    [============================================================] 100%  726.8kB/s   
VNP10A1F.A2023020.h09v05.002.2023097123540.h5.xml
0568/1464: VNP10A1F.A2023020.h09v05.002.2023097123540.h5.xml
  [============================================================] 100%  2.1MB/s   
VNP10A1F.A2023021.h09v04.002.2023097175424.h5
0569/1464: VNP10A1F.A2023021.h09v04.002.2023097175424.h5
  [=========                                                   ]  14%  269.6kB/s    [=================                                           ]  29%  399.1kB/s    [==========================                                  ]  43%  514.3kB/s    [==================================      